In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
! pip install groq
! pip install autogen

In [3]:
# Configure
import os
config_list = [{
    "model": "llama-3.3-70b-versatile",
    "api_key": os.environ.get("GROQ_API_KEY"),
    "api_type": "groq"
}]

In [5]:
from autogen import ConversableAgent, AssistantAgent, UserProxyAgent
from autogen import initiate_chats

agent = ConversableAgent(
    name="chatbot",
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
)

Understanding Autogen Agent Types
Autogen provides specialized agent types for different interaction patterns:

Conversable Agent:

Purpose: Designed for structured conversations, where you need to collect specific pieces of information.

Use Cases: Taking user details like name, employee ID, date of birth, email, etc. Think of form-filling or guided data collection.

Assistant Agent:

Purpose: Task-oriented agents with specific expertise. They act as "assistants" in a workflow.

Use Cases: Generating reports, providing recommendations, performing specific actions based on input.

User Proxy Agent:

Purpose: Acts as an intermediary or "proxy" between the human user and other specialized agents.

Use Cases: Facilitating communication, ensuring smooth transitions between agents, and often handling human input for complex multi-agent workflows.

Think of it as: A bridge between you and the AI system.

In [6]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply['content'])

Here's one:

What do you call a fake noodle?

An impasta!

Hope that made you smile! Do you want to hear another one?


use case :

1. User Registration Agent (Type: Conversable Agent)
2. Sales Recommendation Agent (Type: Assistant Agent)
3. Help Desk Agent (Type: Assistant Agent)
4. Interaction Relay Agent (Type: User Proxy Agent)

In [7]:
# Define the RegistrationAgent for collecting user details
registration_agent = ConversableAgent(
    name="User Registration Agent",
    system_message='''You are a friendly registration assistant.
    Your task is to collect the user's full name, company name, and the product they are interested in.
    Do not ask for additional details. Respond with 'END_SESSION' once all details are obtained.''',
    llm_config={"config_list": config_list},
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [8]:
# Define the HelpDeskAgent for troubleshooting
help_desk_agent = AssistantAgent(
    name="Help Desk Agent",
    system_message='''You are a knowledgeable support assistant.
    Your role is to assist users in buying our product.
    Offer clear steps, share useful resources, and escalate if required.
    Respond with 'END_SESSION' once the issue is addressed.''',
    llm_config={"config_list": config_list},
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [9]:
# Define the SalesAgent for product recommendations
sales_agent = AssistantAgent(
    name="Sales Recommendation Agent",
    system_message='''You are a sales assistant.
    Based on user information, suggest the most suitable product and provide additional details.
    If the user expresses interest, proceed with gathering purchase intent details.
    Respond with 'END_SESSION' when user decides to proceed or not.''',
    llm_config={"config_list": config_list},
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [10]:
# Define the RelayAgent for communication handling
relay_agent = UserProxyAgent(
    name="Interaction Relay Agent",
    system_message='''You act as a bridge between users and specialized agents.
    You do not process requests yourself but ensure smooth communication.''',
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
)

In [11]:
conversation_flow = [
    {
        "sender": registration_agent,
        "recipient": relay_agent,
        "message": "Hi! I'm here to help with your onboarding. Please provide your full name, company, and the product you're interested in.",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Extract user details as JSON: {'full_name': '', 'company': '', 'product': ''}",
        },
        "max_turns": 2,
        "clear_history": True
    },
    {
        "sender": relay_agent,
        "recipient": sales_agent,
        "message": "Based on user details, suggest a suitable product.",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Summarize recommended products as JSON: {'recommended_product': '', 'justification': ''}",
        },
        "max_turns": 3,
        "clear_history": False
    },
    {
        "sender": relay_agent,
        "recipient": help_desk_agent,
        "message": "Based on sale recommendation, closen the conversation with formal sale pitch",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Summarize the assistance provided as JSON: {'resolution_steps': []}",
        },
        "max_turns": 3,
        "clear_history": False
    }
]

In [12]:
# Execute the conversations
session_results = initiate_chats(conversation_flow)
pprint.pp(session_results)


********************************************************************************
Starting a new chat....

********************************************************************************
User Registration Agent (to Interaction Relay Agent):

Hi! I'm here to help with your onboarding. Please provide your full name, company, and the product you're interested in.

--------------------------------------------------------------------------------
Interaction Relay Agent (to User Registration Agent):

hi i am amit kumar from neural network company and i am interested in antivirus product 

--------------------------------------------------------------------------------
User Registration Agent (to Interaction Relay Agent):

Thank you, Amit Kumar from Neural Network Company, for your interest in the antivirus product. 

END_SESSION

--------------------------------------------------------------------------------
Interaction Relay Agent (to User Registration Agent):

yes , i am interested

----

NameError: name 'pprint' is not defined